In [ ]:
# #!/usr/bin/python

import sys
import os
import numpy as np
import random
from random import randint
from random import shuffle
import time
import json
import pickle

import tensorflow as tf
from tensorflow.contrib import rnn

# local packages 
from utils_libs import *
from utils_training import *
# from utils_inference import *
# from mixture_models import *
from pipeline import *

# ------ GPU environment
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# ------ set-up
para_train = {}

# -- path
para_train['arg_py'] = "market1_tar10_len10"
para_train['path_data'] = "../../datasets/bitcoin/market1_tar10_len10/"
para_train['path_model'] = "../../results/volume/m1_t10_1/"
para_train['path_py'] = "../../results/volume/" + para_train['arg_py']
para_train['path_log_error'] = "../../results/volume/log_" + para_train['arg_py'] + ".txt"

# -- data
para_train['para_num_source'] = 4

# -- model
para_train['para_distr_type'] = 'log_normal_logOpt_linearComb' # "log_normal_logOpt_linearComb", 'normal'
para_train['para_var_type'] = "exp" # square, exp
para_train['para_share_type_gate'] = "no_share" # no_share, share, mix
para_train['para_model_type'] = 'linear'

para_train['para_bool_bias_in_mean'] = True
para_train['para_bool_bias_in_var'] = True
para_train['para_bool_bias_in_gate'] = True

if para_train['para_model_type'] == 'rnn':
    para_train['para_x_src_padding'] = False
    para_train['para_add_common_factor'] = False
    para_train['para_common_factor_type'] = "pool" if para_train['para_add_common_factor'] == True else ""
    
elif para_train['para_model_type'] == 'linear':
    para_train['para_x_src_padding'] = True
    para_train['para_add_common_factor'] = False
    para_train['para_common_factor_type'] = "factor" if para_train['para_add_common_factor'] == True else ""

para_train['para_bool_target_seperate'] = False # [Note] if yes, the last source corresponds to the auto-regressive target variable

# -- training
para_train['para_n_epoch'] = 90 # [Note] for sg_mcmc family, "para_n_epoch" could be set to higher values
para_train['para_burn_in_epoch'] = 85
para_train['para_vali_snapshot_num'] = max(1, int(0.05*para_train['para_n_epoch']))
para_train['para_test_snapshot_num'] = 10

para_train['para_hpara_search'] = "random" # random, grid 
para_train['para_hpara_train_trial_num'] = 30
para_train['para_hpara_retrain_num'] = 10
para_train['para_hpara_ensemble_trial_num'] = 3

# -- optimization
para_train['para_loss_type'] = "heter_lk_inv" # "heter_lk_inv"
para_train['para_optimizer'] = "adam" # RMSprop, adam, sgd, adamW, sg_mcmc_RMSprop, sg_mcmc_adam
para_train['para_optimizer_lr_decay_epoch'] = 10 # after the warm-up
para_train['para_optimizer_lr_warmup_epoch'] = max(1, int(0.1*para_train['para_n_epoch']))

para_train['para_early_stop_bool'] = False
para_train['para_early_stop_window'] = 0

para_train['para_validation_metric'] = 'nnllk'
para_train['para_metric_map'] = {'rmse':0, 'mae':1, 'mape':2, 'nnllk':3}

# -- regularization
para_train['para_regu_mean'] = True
para_train['para_regu_var'] = True
para_train['para_regu_gate'] = False

# ----- hpara: hyper parameter ranges

para_hpara_range = {}
para_hpara_range['random'] = {}
para_hpara_range['random']['linear'] = {}
para_hpara_range['random']['rnn'] = {}

# -- linear
if para_train['para_add_common_factor'] == True:
    para_hpara_range['random']['linear']['factor_size'] = [10, 10]
para_hpara_range['random']['linear']['lr'] = [1e-4, 5e-4]
para_hpara_range['random']['linear']['batch_size'] = [10, 300]
# source-wise
# tar1
# para_hpara_range['random']['linear']['l2_mean'] = [1e-1, 5e+1]
# para_hpara_range['random']['linear']['l2_var']  = [1e-1, 5e+1]
# # tar5
# para_hpara_range['random']['linear']['l2_mean'] = [1e-1, 1e+1]
# para_hpara_range['random']['linear']['l2_var']  = [1e-1, 1e+1]
# tar10
para_hpara_range['random']['linear']['l2_mean'] = [1e-1, 5e-0]
para_hpara_range['random']['linear']['l2_var']  = [1e-1, 5e-0]

# # -- rnn
# # source-wise
# para_hpara_range['random']['rnn']['rnn_size'] =  [16, 16]
# para_hpara_range['random']['rnn']['dense_num'] = [0, 3] # inproper value leads to non-convergence in training

# para_hpara_range['random']['rnn']['lr'] = [0.001, 0.001]
# para_hpara_range['random']['rnn']['batch_size'] = [100, 140]

# # source-wise
# para_hpara_range['random']['rnn']['l2_mean'] = [1e-7, 1e-3]
# para_hpara_range['random']['rnn']['l2_var'] = [1e-7, 1e-3]
# if para_regu_gate == True:
#     para_hpara_range['random']['linear']['l2_gate'] = [1e-7, 1e-3]
    
# para_hpara_range['random']['rnn']['dropout_keep_prob'] = [0.7, 1.0]
# para_hpara_range['random']['rnn']['max_norm_cons'] = [0.0, 0.0]

# ----- main process  

if __name__ == '__main__':
    
    # --- data
    src_tr_x, tr_y, src_val_x, val_y, src_ts_x, ts_y = prepare_data(para_train)
    
    # --- run the pipeline
    log_setup(path = para_train['path_log_error'], 
              para_train = para_train,
              para_hpara_range = para_hpara_range)
    
    print('\n --- ', para_train)
    
    train_validate_test(src_tr_x,
                        tr_y,
                        src_val_x,
                        val_y,
                        src_ts_x,
                        ts_y,
                        hyper_para_range = para_hpara_range,
                        para_train = para_train)
    